# **1. Import Library**

Pada tahap ini, Anda perlu mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning.

In [110]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, accuracy_score, f1_score
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from collections import Counter


# **2. Memuat Dataset dari Hasil Clustering**

Memuat dataset hasil clustering dari file CSV ke dalam variabel DataFrame.

In [111]:
# Memuat dataset
file_path = "dataset_inisiasi.csv"
df = pd.read_csv(file_path)

# Menampilkan informasi awal tentang dataset
df.info(), df.head()

# Pastikan kolom fitur dan target ada dalam dataset
print("Kolom dalam dataset:", df.columns)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5224 entries, 0 to 5223
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   provinsi  5224 non-null   object 
 1   jenis     5224 non-null   object 
 2   daerah    5224 non-null   object 
 3   tahun     5224 non-null   float64
 4   periode   5224 non-null   object 
 5   gk        5224 non-null   float64
 6   cluster   5224 non-null   int64  
dtypes: float64(2), int64(1), object(4)
memory usage: 285.8+ KB
Kolom dalam dataset: Index(['provinsi', 'jenis', 'daerah', 'tahun', 'periode', 'gk', 'cluster'], dtype='object')


# **3. Data Splitting**

Tahap Data Splitting bertujuan untuk memisahkan dataset menjadi dua bagian: data latih (training set) dan data uji (test set).

In [112]:
# drop data tidak perlu
df = df.drop(columns=['daerah'])

#splitting data
X = df.drop(columns=['cluster'])
y = df['cluster']

In [113]:
#pipeline preprocessing
categorical_cols = ['provinsi', 'jenis', 'periode']
numeric_cols = ['tahun', 'gk']

preprocessor = ColumnTransformer(transformers=[
    ('num', StandardScaler(), numeric_cols),
    ('cat', OneHotEncoder(drop='first'), categorical_cols)
])

In [114]:
#transformasi data
X_processed = preprocessor.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(
    X_processed, y, test_size=0.2, random_state=42, stratify=y
)

In [115]:
# Tampilkan bentuk (shape) data hasil split
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

# Tampilkan distribusi label (cluster) di train dan test
print("\nDistribusi label pada y_train:", Counter(y_train))
print("Distribusi label pada y_test:", Counter(y_test))

X_train shape: (4179, 39)
X_test shape: (1045, 39)
y_train shape: (4179,)
y_test shape: (1045,)

Distribusi label pada y_train: Counter({1: 1441, 0: 1409, 2: 1329})
Distribusi label pada y_test: Counter({1: 360, 0: 353, 2: 332})


# **4. Membangun Model Klasifikasi**


## **a. Membangun Model Klasifikasi**

Setelah memilih algoritma klasifikasi yang sesuai, langkah selanjutnya adalah melatih model menggunakan data latih.

Berikut adalah rekomendasi tahapannya.
1. Pilih algoritma klasifikasi yang sesuai, seperti Logistic Regression, Decision Tree, Random Forest, atau K-Nearest Neighbors (KNN).
2. Latih model menggunakan data latih.

In [117]:
# === Model 1: Random Forest ===
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)

print("📊 Random Forest Classifier")
print("Accuracy:", accuracy_score(y_test, y_pred_rf))
print("F1 Score:", f1_score(y_test, y_pred_rf, average='weighted'))
print(classification_report(y_test, y_pred_rf))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_rf))

# === Model 2: Logistic Regression ===
lr_model = LogisticRegression(max_iter=1000)
lr_model.fit(X_train, y_train)
y_pred_lr = lr_model.predict(X_test)

print("\n📊 Logistic Regression")
print("Accuracy:", accuracy_score(y_test, y_pred_lr))
print("F1 Score:", f1_score(y_test, y_pred_lr, average='weighted'))
print(classification_report(y_test, y_pred_lr))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_lr))

📊 Random Forest Classifier
Accuracy: 1.0
F1 Score: 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       353
           1       1.00      1.00      1.00       360
           2       1.00      1.00      1.00       332

    accuracy                           1.00      1045
   macro avg       1.00      1.00      1.00      1045
weighted avg       1.00      1.00      1.00      1045

Confusion Matrix:
 [[353   0   0]
 [  0 360   0]
 [  0   0 332]]

📊 Logistic Regression
Accuracy: 1.0
F1 Score: 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       353
           1       1.00      1.00      1.00       360
           2       1.00      1.00      1.00       332

    accuracy                           1.00      1045
   macro avg       1.00      1.00      1.00      1045
weighted avg       1.00      1.00      1.00      1045

Confusion Matrix:
 [[353   0   0]
 [  0 360   0]
 [  0   0 332]]


In [118]:
# Inisialisasi dan latih model Logistic Regression
log_model = LogisticRegression(max_iter=1000, random_state=42)
log_model.fit(X_train, y_train)

LogisticRegression(max_iter=1000, random_state=42)



**terlihat Logistic regression dan random forest mendapat nilai 1.0.**\
dimana itu artinya musngkin terdapat overfitting.\
Jadi saya memilih random forest dan logistic regression untuk dituning lagi.

## **b. Evaluasi Model Klasifikasi**

Berikut adalah **rekomendasi** tahapannya.
1. Lakukan prediksi menggunakan data uji.
2. Hitung metrik evaluasi seperti Accuracy dan F1-Score (Opsional: Precision dan Recall).
3. Buat confusion matrix untuk melihat detail prediksi benar dan salah.

In [119]:
# Dictionary model yang sudah dilatih sebelumnya
models = {
    'Random Forest': rf_model,
    'Logistic Regression': log_model
}

# Evaluasi setiap model lebih lanjut
evaluation_results = {}
confusion_matrices = {}

for name, model in models.items():
    y_pred = model.predict(X_test)  # Prediksi data uji
    
    # Menghitung metrik evaluasi
    report = classification_report(y_test, y_pred, output_dict=True)
    evaluation_results[name] = report
    
    # Menghitung confusion matrix
    confusion_matrices[name] = confusion_matrix(y_test, y_pred)

# Menampilkan hasil evaluasi model
for name in evaluation_results:
    print(f"\n📊 {name}")
    print("Accuracy:", round(evaluation_results[name]['accuracy'], 4))
    print("F1 Score:", round(evaluation_results[name]['weighted avg']['f1-score'], 4))
    print("Classification Report:")
    print(classification_report(y_test, models[name].predict(X_test)))
    print("Confusion Matrix:")
    print(confusion_matrices[name])


📊 Random Forest
Accuracy: 1.0
F1 Score: 1.0
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       353
           1       1.00      1.00      1.00       360
           2       1.00      1.00      1.00       332

    accuracy                           1.00      1045
   macro avg       1.00      1.00      1.00      1045
weighted avg       1.00      1.00      1.00      1045

Confusion Matrix:
[[353   0   0]
 [  0 360   0]
 [  0   0 332]]

📊 Logistic Regression
Accuracy: 1.0
F1 Score: 1.0
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       353
           1       1.00      1.00      1.00       360
           2       1.00      1.00      1.00       332

    accuracy                           1.00      1045
   macro avg       1.00      1.00      1.00      1045
weighted avg       1.00      1.00      1.00      1045

Confusion Matrix:
[[353   0   0]

Tulis hasil evaluasi algoritma yang digunakan, jika Anda menggunakan 2 algoritma, maka bandingkan hasilnya.

1. Random forest classifier
    - Accuracy: 1.00 (100%)
    - F1 Score (weighted avg): 1.00
    - Random Forest mampu mengklasifikasikan semua data uji dengan presisi dan akurasi sempurna.
    - Tidak ada satupun data yang salah klasifikasi (semua nilai di luar diagonal = 0).
    - Ini mengindikasikan model bisa jadi mengalami overfitting terhadap data pelatihan.

2. Logistic Regression
    - Accuracy: 1.00 (100%)
    - F1 Score (weighted avg): 1.00
    - Seperti Random Forest, Logistic Regression juga menunjukkan kinerja yang sempurna pada data uji.
    - Hal ini cukup tidak biasa karena Logistic Regression biasanya tidak terlalu kompleks dan jarang memberikan akurasi setinggi itu, apalagi jika dataset kompleks.

## **c. Tuning Model Klasifikasi (Optional)**

Gunakan GridSearchCV, RandomizedSearchCV, atau metode lainnya untuk mencari kombinasi hyperparameter terbaik

In [120]:
# Inisialisasi model dasar
rf = RandomForestClassifier(random_state=42)

# Tentukan parameter grid yang ingin diuji
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2']
}

# GridSearchCV setup
grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    scoring='f1_macro',
    n_jobs=-1,  # Gunakan semua core
    verbose=2
)

# Jalankan pencarian
grid_search.fit(X_train, y_train)

# Tampilkan hasil terbaik
print("Best Parameters:\n", grid_search.best_params_)
print("\nBest F1 Score:", grid_search.best_score_)

# Model terbaik
best_rf_model = grid_search.best_estimator_

Fitting 5 folds for each of 216 candidates, totalling 1080 fits
Best Parameters:
 {'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}

Best F1 Score: 1.0


## **d. Evaluasi Model Klasifikasi setelah Tuning (Optional)**

Berikut adalah rekomendasi tahapannya.
1. Gunakan model dengan hyperparameter terbaik.
2. Hitung ulang metrik evaluasi untuk melihat apakah ada peningkatan performa.

In [121]:
# Prediksi data uji dengan model terbaik hasil tuning
y_pred_best = best_rf_model.predict(X_test)

# Evaluasi performa
accuracy_best = accuracy_score(y_test, y_pred_best)
f1_best = f1_score(y_test, y_pred_best, average='weighted')
report_best = classification_report(y_test, y_pred_best)
conf_matrix_best = confusion_matrix(y_test, y_pred_best)

# Tampilkan hasil evaluasi
print("📈 Evaluasi Model setelah Tuning (RandomForest - GridSearchCV):")
print(f"Accuracy: {accuracy_best:.4f}")
print(f"F1 Score (weighted): {f1_best:.4f}\n")
print("Classification Report:")
print(report_best)
print("Confusion Matrix:")
print(conf_matrix_best)

📈 Evaluasi Model setelah Tuning (RandomForest - GridSearchCV):
Accuracy: 1.0000
F1 Score (weighted): 1.0000

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       353
           1       1.00      1.00      1.00       360
           2       1.00      1.00      1.00       332

    accuracy                           1.00      1045
   macro avg       1.00      1.00      1.00      1045
weighted avg       1.00      1.00      1.00      1045

Confusion Matrix:
[[353   0   0]
 [  0 360   0]
 [  0   0 332]]


## **e. Analisis Hasil Evaluasi Model Klasifikasi**

Analisis Hasil Evaluasi Model Random Forest dan Logistic Regression
Berdasarkan hasil evaluasi, berikut analisisnya:

-  Akurasi setelah tuning: 100% → Model menunjukkan performa sempurna dalam klasifikasi.
- Precision, Recall, dan F1-Score:
Semua metrik bernilai 1.00 untuk ketiga kelas (0, 1, 2), baik sebelum maupun sesudah tuning.
- Confusion Matrix:
Menunjukkan tidak ada kesalahan prediksi. Semua data uji berhasil diklasifikasikan dengan benar oleh kedua model (nilai di luar diagonal = 0).

 **rekomendasi** tahapannya.
1. Perbandingan Hasil Evaluasi Sebelum dan Setelah Tuning
   - Akurasi sebelum tuning: 100%
   - Akurasi setelah tuning: 100% (tidak ada perubahan)
   - Precision sebelum tuning: 1.00 (untuk semua kelas)
   - Precision setelah tuning: 1.00 (tidak ada perubahan)
   - Recall sebelum tuning: 1.00 (untuk semua kelas)
   - Recall setelah tuning: 1.00 (tidak ada perubahan)
   - F1-Score sebelum tuning: 1.00 (untuk semua kelas)
   - F1-Score setelah tuning: 1.00 (tidak ada perubahan)

Kesimpulan:
Model sudah mencapai performa maksimal bahkan sebelum tuning. GridSearchCV tidak memberikan peningkatan karena akurasi dan metrik lainnya sudah sempurna.

2. Identifikasi kelemahan model, seperti:\
   **Precision atau Recall rendah untuk kelas tertentu:**
   - Tidak ada kelemahan terkait precision atau recall → semua nilai = 1.00.
   - Logistic Regression seharusnya tidak seakurat itu jika data benar-benar kompleks → ini mencurigakan.

   **Overfitting atau Underfitting?**
   - Kemungkinan besar model mengalami overfitting, karena;
   - Kedua model (Random Forest & Logistic Regression) menunjukkan akurasi 100% bahkan pada data uji.
   - KNN mungkin tidak optimal untuk dataset ini, terutama jika ada banyak fitur yang kurang relevan atau terdapat noise.

3. Rekomendasi Tindakan Lanjutan
- Validasi Model dengan Cross-Validation, Gunakan Stratified K-Fold untuk mengecek apakah akurasi tetap konsisten di berbagai pembagian data.
- Visualisasi Feature Importance (untuk Random Forest), Untuk mengecek apakah ada fitur dominan yang menyebabkan model terlalu mudah mengklasifikasi data.
-Uji dengan Data Baru (Jika Ada), Coba uji model dengan dataset dari luar (unseen data) untuk menguji generalisasi model.
-  Feature Engineering Tambahan, Eksplorasi lebih lanjut apakah ada fitur tambahan yang bisa membantu pemisahan kelas secara lebih alami (bukan karena dominasi satu fitur).
- Coba Algoritma Lain (Opsional), Untuk eksplorasi lebih lanjut, model seperti XGBoost atau SVM bisa dicoba untuk pembanding performa dan perilaku.
\\
